In [1]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

setup geometry

In [2]:
geo = SplineGeometry()
geo.AddRectangle( (-3,-2), (3, 2), bcs = ("top", "out", "bot", "in"))
geo.AddCircle ( (0, 0), r=0.5, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.2))
mesh.Curve(3);
#Draw(mesh)

setup FEM space

In [3]:
# viscosity
nu = 0.001
# Order of spaces
k = 2
# H1 vs VectorH1 -> vector field?!
V = VectorH1(mesh,order=k, dirichlet="top|bot|cyl|in|out")
Q = H1(mesh,order=k-1)
X = FESpace([V,Q]) # X = [V,V,Q] (without VectorH1)

setup bilinear form

In [4]:
u,p = X.TrialFunction()
v,q = X.TestFunction()

a = BilinearForm(X)
a += (InnerProduct(grad(u),grad(v))+div(u)*q+div(v)*p)*dx
a.Assemble()

setup boundary conditions (?)

In [5]:
gfu = GridFunction(X)
# setup flow condition
uinf = 0.001
uin = CoefficientFunction((uinf,0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("in|top|bot|out"))

velocity = CoefficientFunction(gfu.components[0])
scene = Draw(velocity, mesh, "vel")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

solve equation

In [6]:
res = gfu.vec.CreateVector()
res.data = -a.mat * gfu.vec
inv = a.mat.Inverse(X.FreeDofs())
gfu.vec.data += inv * res
scene.Redraw()

Drag

$J(\Omega ) = \frac{1}{2} \int_\Omega Du : Du dx$

In [13]:
vel = gfu.vec.data
drag = 0.5*InnerProduct(vel,vel)
print(drag)
#d = 0.5*InnerProduct(grad(u)[0],grad(u)[0])*dx
#print(d)

2.112761569110374


### Shape derivative
$S_1 = \biggl(\frac{1}{2}Du : Du - p div(u) \biggr) I_2 + Du^T p - Du^T Du.$

In [98]:
l = (0.5*InnerProduct(grad(u),grad(u))-p*div(v))*Id(2)
r = grad(u).trans*p-grad(u).trans*grad(u)
S = l+r

In [99]:
r = X.TrialFunction()
s = X.TestFunction()
gf = GridFunction(X)

In [ ]:
# gf, gfp: solution to state&adjoint eq

In [107]:
def Cost(u):
    return 0.5*InnerProduct(grad(u),grad(u))*dx

def Equation(u,w):
    return (InnerProduct(grad(u),grad(v))+div(u)*q-div(v)*p)*dx

G_pde = Cost(gfu) * Equation(gfu, gfp)

W = V.TestFunction() # X. or V.
dJOmega = LinearForm(VEC)
dfOmega += G_pdre.DiffShape(W)

NgException: Operator "grad" does not exist for CompoundFESpace!

In [71]:
"""
grad_S = CoefficientFunction((S.Diff(x),S.Diff(y)))
VEC = H1(mesh, order=1, dim=2)
# grid function for deformation field
gfset = GridFunction(VEC)
gfX = GridFunction(VEC)
scene2 = Draw(gfset)

# Test and trial functions
PHI, PSI = VEC.TnT()

# shape derivative
dJOmega = LinearForm(VEC)
dJOmega += (div(PSI)*f + InnerProduct(grad_f, PSI) )*dx
"""

'\ngrad_S = CoefficientFunction((S.Diff(x),S.Diff(y)))\nVEC = H1(mesh, order=1, dim=2)\n# grid function for deformation field\ngfset = GridFunction(VEC)\ngfX = GridFunction(VEC)\nscene2 = Draw(gfset)\n\n# Test and trial functions\nPHI, PSI = VEC.TnT()\n\n# shape derivative\ndJOmega = LinearForm(VEC)\ndJOmega += (div(PSI)*f + InnerProduct(grad_f, PSI) )*dx\n'